In [1]:
from Bio import SeqIO
from Bio.Seq import MutableSeq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import generic_dna
import csv

ModuleNotFoundError: No module named 'Bio'

In [2]:
refseq = SeqIO.read('LC300_genome.gb', 'genbank')

In [3]:
refseq

SeqRecord(seq=Seq('CACATTATTTGACGACGACGAGCGGGGAAACGGCGGCCATCATTTTCCGCCATA...AGC', IUPACAmbiguousDNA()), id='CP008903.1', name='CP008903', description='Geobacillus sp. LC300, complete genome', dbxrefs=['BioProject:PRJNA254417', 'BioSample:SAMN02903857'])

In [6]:
changes_dict = {}  # Contains all indentified coding snps from LC300 with high abundance
with open('genetic_changes.csv') as infile:  # snps.csv contains the identified indels in the LC300 genome
    changes = csv.reader(infile, delimiter = ',')  
    header_data = next(changes)
    for protid, function, position, origseq, variant in changes:
        changes_dict[protid] = function, int(position), origseq, variant  # Saves snp info for each protid. how to handle multiples? Not used except for the keys right now.

In [7]:
print (changes_dict)

{'AKU25193.1': (' hypothetical protein', 14662, ' C', ' CG'), 'AKU25520.1': (' hypothetical protein', 464445, ' C', ' CG'), 'AKU25531.1': (' glycerol-3-phosphate dehydrogenase', 487424, ' T', ' TC'), 'AKU25539.1': (' transposase', 496187, ' TA', ' T'), 'AKU25552.1': (' methylglyoxal synthase', 514436, ' T', ' TC'), 'AKU25556.1': (' hypothetical protein', 519820, ' C', ' CT'), 'AKU25679.1': (' hypothetical protein', 661900, ' G', ' GC'), 'AKU25779.1': (' metal ABC transporter permease', 767932, ' G', ' GC'), 'AKU26206.1': (' hypothetical protein', 1262314, ' TC', ' T'), 'AKU26253.1': (' hypothetical protein', 1310368, ' A', ' AG'), 'AKU26310.1': (' hypothetical protein', 1372460, ' A', ' AG'), 'AKU26719.1': (' hypothetical protein', 1877616, ' C', ' CG'), 'AKU26900.1': (' hypothetical protein', 2080219, ' T', ' TA'), 'AKU27200.1': (' hypothetical protein', 2490953, ' T', ' TC'), 'AKU27233.1': (' hypothetical protein', 2523607, ' G', ' GC'), 'AKU27296.1': (' LacI family transcriptional r

In [11]:
def printSequence(dnaseq, feature, additionalBases):
    # Gets DNA and protein sequence of a given feature
    # dnaseq: DNA sequence to extract feature coding sequence from
    # feature: A biopython SeqFeature object 
    # additionalBases: Number of additional bases downstream of the CDS to include in the extracted sequence. Used
    #                  to identify extended CDSs due to frameshifts resulting from SNPs.
    
    
    loc = str(feature.location) # Gets location of CDS
    loc = loc[1:-4]  # Removes junk surrounding sequence location
    start, end = loc.split(':')  # gets start and end position of CDS
    # Cast to int for indexing in the genome sequence
    
    # Adds additional bp downstream of coding stop to account for possible extended CDS due to frameshift
    if '-' in str(feature.location):         
        start = int(start) -additionalBases  
        end = int(end)
    else: 
        start = int(start)
        end = int(end) + additionalBases
    
    protid = str(feature.qualifiers['protein_id']).lstrip('[\'').rstrip('\']')
    print('Protein id: ' + protid)
    
    print('Protein: ' + str(feature.qualifiers['product']))
    
    print('DNA sequence: ' + dnaseq[start:end])
    
    translation = str(feature.qualifiers['translation']).lstrip('[\'').rstrip('\']')
    print('AA sequence: ' + translation)
    #print(dnaseq[feature.location])
    
    print('DNA seq start: ' + str(start))
    print('DNA seq end: ' + str(end))
    
    print(' ')


In [12]:
for feature in refseq.features:  # Finds all sequences with identified snps
    if 'CDS' in feature.type:
        protid = str(feature.qualifiers['protein_id'])[2:-2]  # Strips redundant characters to get prot id in correct format.
        
        #  Finds and extracts sequence information from the features that exist in snp_dict:
        if protid in changes_dict:
            printSequence(refseq.seq, feature, 500)  # The printed sequences below are copied manually to a file for analysis

Protein id: AKU25193.1
Protein: ['hypothetical protein']
DNA sequence: ACGAATGCGGGTGGTGGCGGGGGCAGGGCATGGCTGGTCACGATAGGGGGATACAATGAAACAGCATCGGTTGTCGTTTGAATTGGAAATGGTTGTAGAACGCGCATACGAATGCATCAAGCTAACTGGACGGCTCGCCTATGATACGCAGCTCGCGGCGGAACAAAAGCTGGCGACGGCGGTGGCTGCTGTCAAGCGCCGCCTTGTCGTTGTCGACGTGAGCGGCCTTCGATTTTTGGACAGTACTGGATTAGCGCTCCTTGTCCGCTTTTTTAAGCAAATCGTCAGCGAAAGCCGCGCCATGGCCATCGTCGTCCGGGATAACGCCGCTATCGAAAAGATTTTGCGGATGGCCAAGCTCGACCGGCTGTTGCCGATCGTTGGCGATGAACAACAGCTGTTGTCCCTTGCGCCGCCAAGCTCTTTCGACCATATGAGCCAGGAGGCGCTTTATTCGGCATGGCGTCAGCGTTCGTCATAGACATGGCTCTCATGCCGAATCAACTGGAAGATTTCACATTTCGCTGGCCCAAAAGCGTGTCGTCTATTTTTCCGACTGCTGAAGGCAAGCCACAGGCTTGCCTTCGTCGCGCCAAGGTGTGACGGAAGACGGAATAACCACCTGCTTCCCAGACCCATGCATGGGTCTGGAAGCGGTGTTGTTCAGTCACCCGCCAGTCGGGCATCTCGATCGAAACGACTGAACAGACCAGTTGTTCCCCTATTGCCGCCGATGGCCCCCGAGCGATGAGACATTCGTTCGGGGTTTTGTTCCTCTTGGCCGGGAGCATCCCTGCTTCTGCAAATGCATTCTCATTGTCAAGTGAACTACCCCCACTTAATTTTCTAGCGAAAATTTGAAGTGGGGGCTTCCAAAGAAGTTTGACTGCTTCAAGCAA
AA sequence: MLEAVKLLWKPPLQIF

In [13]:
for feature in refseq.features:  # Finds all sequences with identified snps
    if 'CDS' in feature.type:
        protid = str(feature.qualifiers['protein_id'])[2:-2]  # Strips redundant characters to get prot id in correct format.
        
        #  Finds and extracts sequence information from the features that exist in snp_dict:
        if protid in snp_dict:
            printSequence(refseq.seq, feature, 500)  # The printed sequences below are copied manually to

Protein id: AKU25193.1
Protein: ['hypothetical protein']
DNA sequence: ACGAATGCGGGTGGTGGCGGGGGCAGGGCATGGCTGGTCACGATAGGGGGATACAATGAAACAGCATCGGTTGTCGTTTGAATTGGAAATGGTTGTAGAACGCGCATACGAATGCATCAAGCTAACTGGACGGCTCGCCTATGATACGCAGCTCGCGGCGGAACAAAAGCTGGCGACGGCGGTGGCTGCTGTCAAGCGCCGCCTTGTCGTTGTCGACGTGAGCGGCCTTCGATTTTTGGACAGTACTGGATTAGCGCTCCTTGTCCGCTTTTTTAAGCAAATCGTCAGCGAAAGCCGCGCCATGGCCATCGTCGTCCGGGATAACGCCGCTATCGAAAAGATTTTGCGGATGGCCAAGCTCGACCGGCTGTTGCCGATCGTTGGCGATGAACAACAGCTGTTGTCCCTTGCGCCGCCAAGCTCTTTCGACCATATGAGCCAGGAGGCGCTTTATTCGGCATGGCGTCAGCGTTCGTCATAGACATGGCTCTCATGCCGAATCAACTGGAAGATTTCACATTTCGCTGGCCCAAAAGCGTGTCGTCTATTTTTCCGACTGCTGAAGGCAAGCCACAGGCTTGCCTTCGTCGCGCCAAGGTGTGACGGAAGACGGAATAACCACCTGCTTCCCAGACCCATGCATGGGTCTGGAAGCGGTGTTGTTCAGTCACCCGCCAGTCGGGCATCTCGATCGAAACGACTGAACAGACCAGTTGTTCCCCTATTGCCGCCGATGGCCCCCGAGCGATGAGACATTCGTTCGGGGTTTTGTTCCTCTTGGCCGGGAGCATCCCTGCTTCTGCAAATGCATTCTCATTGTCAAGTGAACTACCCCCACTTAATTTTCTAGCGAAAATTTGAAGTGGGGGCTTCCAAAGAAGTTTGACTGCTTCAAGCAA
AA sequence: MLEAVKLLWKPPLQIF

In [15]:
# Extracts an extended sequence for AKU26253.1, since the new CDS extends beyond the initially added 500 bp
for feature in refseq.features:  # Finds all sequences with identified snps
    if 'CDS' in feature.type:
        protid = str(feature.qualifiers['protein_id'])[2:-2]  # Strips redundant characters to get prot id in correct format.
        
        #  Finds and extracts sequence information from the features that exist in snp_dict:
        if protid == 'AKU26253.1':
            printSequence(refseq.seq, feature, 1500)  # The printed sequences below are copied manually to

Protein id: AKU26253.1
Protein: ['hypothetical protein']
DNA sequence: ATGAAACATGATCATATTTCCTTTAAAGAGTATACCATGGACAACCTCTCTTTGCCAAGCAACATCGCCGATCTCATTCCACGGGATAACATGGCCCACGTGGTTCATGAGATGGTTGAACGCATCCCGATGGAGACGTTTCTTGCTTACTACAAAGGAGGGGCGCCTCCGCCTATCATCCGAAAATGATGACCAAAATTCTCCTTTACGCTTACACCCAAAAAATGTATCATGGCCGTGAGATTGCACGGCAGCTCGAAGTCCACCTTCCCCTCATGTGGCTAAGTGGATTTCAAAAGCCGGACTTCCGCTCCATCAATCGGTTTCGTTCGGAGCGGATGAAAGACCTGATTGACGACCTGTTCCGGGAAATGATCACGCTGCTCGTGGCCGACGGCTATGTCAAGATGGAAGATTATTTTGTGGATGGAACGAAAATTGAAGCGAATGCCAACCGGTACACCTTCGTTTGGCGCAAATCGGCTGAGAAGTACCAGGAGAAACTCCAAGCCAATGTCGATCGGCTGATTGCCCAGATCGAGGCGATCGTGGAAGAAGAAAAGGCGGAAGACATCGACCCTTCGCCGGCCTTTACGTCAGAAGAAATCCGAAAGAAAACCGAGGAGTGGGAAAAACGGTTGGAGGCCGAGCCGGACAACCAACCGTTGAAGAAGGCCATCAAGAAGATGAAGGAGGACTACTTGCCCCGCAGTGAAAAGTATGAACACCAACTCCATGTATGCGGGGATCGCAACAGCTATTCCAAGACCGATGTGGATGCCACCTTCATGCGGTTGAAGGAGGATCACATGCGAAACGGCCAGCTCAAGCCGGCCTATAACGTACAGGTGGGTTCTTCCGGCCAATTTATTTTGGGGTATTCCCTTCATCAGAGGCCGGGCGACACTCGTTGTCTTCTCCCGCATTTG